In [1]:
!pip install diffusers
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.8 MB/s eta 0:00:00


In [2]:
HEIGHT = 512
WIDTH = 512
FPS = 10
NUM_FRAMES = 10


from torch.nn import functional as F
from PIL import Image, ImageFilter
import torch
import numpy as np
import imageio
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import files
from diffusers import I2VGenXLPipeline
from diffusers.utils import load_image, export_to_gif

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/drive


In [3]:
#helper function

def warmup(model):
  prompt = "Classical art, A curious fox explores a golden-hued forest bathed in the warm light of autumn, feeling playful and mischievous, 4k, high quality"
  img1 = np.zeros((512, 512, 3) , dtype=np.uint8)
  img1 = Image.fromarray(img1)
  img2 = img1

def initialize_i2vgen():
  repo_id = "ali-vilab/i2vgen-xl"
  pipeline = I2VGenXLPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, variant="fp16").to("cuda")
  return pipeline

def generate_vid(pipe,image,prompt,height,width,fps,num_frames):
  generator = torch.manual_seed(8888)
  frames = pipe(
      prompt=prompt,
      image=image,
      # height=height,
      # width=width,
      generator=generator,
      target_fps = fps,
      num_frames = num_frames
  ).frames[0]
  return frames


pipe = initialize_i2vgen()

In [5]:
image_url = ""
image = load_image(image_url).convert("RGB")
prompt = ""
result = generate_vid(pipe,image,prompt,HEIGHT,WIDTH,FPS,10)
frames = np.array(result)


  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
pil_images = []
for i in range(frames.shape[0]):
    frame = frames[i]
    pil_image = Image.fromarray(frame)
    pil_images.append(pil_image)



In [7]:
imageio.mimsave("temp4.gif", pil_images,format='GIF', fps= FPS)
!cp temp4.gif /content/drive/MyDrive/AI/